In [52]:
import pandas as pd
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 5]
import seaborn as sns
from scipy import stats
import math

**Гипотеза о равенстве вероятностей «успеха».**

In [53]:
def valueStatsCriteria(n11, n12, n21, n22):
    n1_ = n11 + n12
    n2_ = n21 + n22

    n_1 = n11 + n21
    n_2 = n12 + n22

    n = n1_ + n2_

    return (n11 / n1_ - n21 / n2_) / math.sqrt(n_1 / n * (1 - n_1 / n) * (1 / n1_ + 1 / n2_))

In [54]:
def checkValueStatsCriteria(a, value, n, Ha="p1 != p2"):
    pvalue = None
    if Ha == "p1 != p2":
        tmin = stats.t.ppf(a / 2, n)
        tmax = stats.t.ppf(1 - a / 2, n)
        pvalue = 2 * stats.norm.cdf(value)
        pvalue2 = 2 - 2 * stats.norm.cdf(value)
        if pvalue > pvalue2: pvalue = pvalue2
        if value > tmin and value < tmax: return ("H0 (p1 = p2) - accept", "PValue={}".format(pvalue))
    elif Ha == "p1 > p2":
        tmax = stats.t.ppf(1 - a, n)
        pvalue = 1 - stats.norm.cdf(value)
        if value < tmax: return ("H0 (p1 = p2) - accept", "PValue={}".format(pvalue))
    elif Ha == "p1 < p2":
        tmin = stats.t.ppf(a, n)
        pvalue = stats.norm.cdf(value)
        if value > tmin: return ("H0 (p1 = p2) - accept", "PValue={}".format(pvalue))

    return ("H0 (p1 = p2) - reject", "PValue={}".format(pvalue))

**Задача 3.1**

In [55]:
n11 = 42; n12 = 63
n21 = 65; n22 = 130

# p1 - вероятность голосования за кандидата А в Курске
# p2 - вероятность голосования за кандидата А во Владивостоке

T = valueStatsCriteria(n11, n12, n21, n22)
print("value stats criteria -", T)

checkValueStatsCriteria(0.05, T, n11 + n12 + n21 + n22, Ha="p1 != p2")

value stats criteria - 1.1497694901123454


('H0 (p1 = p2) - accept', 'PValue=0.2502388245506466')

На уровне значимости 0,05 можно считать, что уровни поддержки кандидата А в Курске и Владивостоке одинаковы.

**Задача 3.2**

In [56]:
n11 = 3; n12 = 172
n21 = 32; n22 = 168

# p1 - вероятность заболеть среди принимавших лекарство
# p2 - вероятность заболеть среди не принимавших лекарство

T = valueStatsCriteria(n11, n12, n21, n22)
print("value stats criteria -", T)

checkValueStatsCriteria(0.05, T, n11 + n12 + n21 + n22, Ha="p1 < p2")

value stats criteria - -4.74436545825051


('H0 (p1 = p2) - reject', 'PValue=1.045804697525518e-06')

На уровне значимости 0,05 можно считать, что лекарство эффективно.

**Критерий Манна-Уитни-Уилкоксона.**

In [57]:
def valueStatsCriteria_(sequence1, sequence2):
    maxvalue = 10000000000
    ranks1 = [0] * len(sequence1)
    ranks2 = [0] * len(sequence2)

    temp_sequence1 = sequence1[:]
    temp_sequence2 = sequence2[:]

    rank = 1
    while True:
        minval1 = min(temp_sequence1)
        idx1 = temp_sequence1.index(minval1)

        minval2 = min(temp_sequence2)
        idx2 = temp_sequence2.index(minval2)

        if minval1 == maxvalue and minval2 == maxvalue: break

        if minval1 == minval2:
            ranks1[idx1] = rank + 0.5
            ranks2[idx2] = rank + 0.5
            temp_sequence1[idx1] = maxvalue
            temp_sequence2[idx2] = maxvalue
            rank += 2        
        elif minval1 < minval2:
            ranks1[idx1] = rank
            temp_sequence1[idx1] = maxvalue
            rank += 1
        else:
            ranks2[idx2] = rank
            temp_sequence2[idx2] = maxvalue
            rank += 1

    W = sum(ranks2)
    M = len(ranks2) / 2 * (len(ranks1) + len(ranks2) + 1)
    D = len(ranks1) * len(ranks2) / 12 * (len(ranks1) + len(ranks2) + 1)

    return (W - M) / math.sqrt(D)

In [58]:
def checkValueStatsCriteria_(a, value, n, Ha="Q != 0"):
    pvalue = None
    if Ha == "Q != 0":
        tmin = stats.t.ppf(a / 2, n)
        tmax = stats.t.ppf(1 - a / 2, n)
        pvalue = 2 * stats.norm.cdf(value)
        pvalue2 = 2 - 2 * stats.norm.cdf(value)
        if pvalue > pvalue2: pvalue = pvalue2
        if value > tmin and value < tmax: return ("H0 (Q = 0) - accept", "PValue={}".format(pvalue))
    elif Ha == "Q > 0":
        tmax = stats.t.ppf(1 - a, n)
        pvalue = 1 - stats.norm.cdf(value)
        if value < tmax: return ("H0 (Q = 0) - accept", "PValue={}".format(pvalue))
    elif Ha == "Q < 0":
        tmin = stats.t.ppf(a, n)
        pvalue = stats.norm.cdf(value)
        if value > tmin: return ("H0 (Q = 0) - accept", "PValue={}".format(pvalue))

    return ("H0 (Q = 0) - reject", "PValue={}".format(pvalue))

**Задача 3.3**

In [68]:
sequence1 = [130, 110, 120, 140, 200, 130, 140, 170, 160, 140]
sequence2 = [120, 190, 130, 160, 150, 120, 110, 120, 200]

W = valueStatsCriteria_(sequence1, sequence2)
print("value stats criteria -", W)

checkValueStatsCriteria_(0.05, W, len(sequence1) + len(sequence2), Ha="Q < 0")

value stats criteria - -0.2857738033247041


('H0 (Q = 0) - accept', 'PValue=0.3875256868350412')

На уровне значимости до 0.387 можно считать, горожане в среднем не имеют
более высокое давление чем жители сельской местности

**Задача 3.4**

In [64]:
sequence1 = [102.4, 100.0, 67.6, 65.9, 64.7, 39.6, 31.2]
sequence2 = [48.1, 45.5, 41.7, 35.4, 29.1, 18.9, 58.3, 68.8, 71.3, 94.3]

W = valueStatsCriteria_(sequence1, sequence2)
print("value stats criteria -", W)

checkValueStatsCriteria_(0.05, W, len(sequence1) + len(sequence2), Ha="Q != 0")

value stats criteria - -1.0734900802433864


('H0 (Q = 0) - accept', 'PValue=0.28305128705188465')

с надёжностью 0,95 можно считать, что уровень
гистамина у склонных и не склонных к аллергии курильщиков значимо не различается